In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Skip this cell if you already have a chunk of the transactions data.

In [2]:
file_name = '../data/transactions.csv'
chunksize = 10 ** 5
chunk_iter = pd.read_csv(file_name, chunksize=chunksize)
chunk1 = chunk_iter.next()
chunk1.to_csv('../data/transactions_chunk1.csv', index=False)

In [3]:
lines = sc.textFile('../data/transactions_chunk1.csv')

In [4]:
header = lines.first()
lines = lines.filter(lambda row: row != header)

In [5]:
# Functions for parsing the strings from the CSV file

def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None

def toFloatSafe(inval):
  try:
    return float(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split(',')
  return Row(
    int(r[0]),         # Don't want this column to be nullable
    toLongSafe(r[1]),
    toLongSafe(r[2]),
    toLongSafe(r[3]),
    toLongSafe(r[4]),
    toLongSafe(r[5]),
    toTimeSafe(r[6]),
    toFloatSafe(r[7]),
    r[8],
    toLongSafe(r[9]),
    toFloatSafe(r[10]))  

In [6]:
transactions_schema = StructType([
    StructField("id", LongType(), False),
    StructField("chain", LongType(), True),
    StructField("dept", LongType(), True),
    StructField("category", LongType(), True),
    StructField("company", LongType(), True),
    StructField("brand", LongType(), True),
    StructField("date", DateType(), True),
    StructField("productsize", DoubleType(), True),
    StructField("purchasemeasure", StringType(), True),
    StructField("purchasequantity", LongType(), True),
    StructField("purchaseamount", DoubleType(), True)
    ])

In [7]:
rowRDD = lines.map(lambda p: stringToPost(p))

In [8]:
transactions = sqlContext.createDataFrame(rowRDD, transactions_schema)

In [9]:
transactions.printSchema()

root
 |-- id: long (nullable = false)
 |-- chain: long (nullable = true)
 |-- dept: long (nullable = true)
 |-- category: long (nullable = true)
 |-- company: long (nullable = true)
 |-- brand: long (nullable = true)
 |-- date: date (nullable = true)
 |-- productsize: double (nullable = true)
 |-- purchasemeasure: string (nullable = true)
 |-- purchasequantity: long (nullable = true)
 |-- purchaseamount: double (nullable = true)



In [10]:
transactions.show()

+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|   id|chain|dept|category|   company|brand|      date|productsize|purchasemeasure|purchasequantity|purchaseamount|
+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|86246|  205|   7|     707|1078778070|12564|2012-03-02|       12.0|             OZ|               1|          7.59|
|86246|  205|  63|    6319| 107654575|17876|2012-03-02|       64.0|             OZ|               1|          1.59|
|86246|  205|  97|    9753|1022027929|    0|2012-03-02|        1.0|             CT|               1|          5.99|
|86246|  205|  25|    2509| 107996777|31373|2012-03-02|       16.0|             OZ|               1|          1.99|
|86246|  205|  55|    5555| 107684070|32094|2012-03-02|       16.0|             OZ|               2|         10.38|
|86246|  205|  97|    9753|1021015020|    0|2012-03-02|        1.0|     

Get number of refund transactions

In [11]:
returns = transactions.select('id',
                              F.when(transactions['purchaseamount'] < 0, 1).otherwise(0).alias('return'))\
                      .withColumn('1', F.lit(1))
returns = returns.groupBy('id').agg(F.sum('1').alias('total_trans'), F.sum("return").alias('total_returns'))
returns.show()

+--------+-----------+-------------+
|      id|total_trans|total_returns|
+--------+-----------+-------------+
|18470775|        350|            8|
|14723452|        755|            5|
|15738658|         39|            0|
|17552659|        591|            4|
|12996040|        326|            5|
|16078766|        966|           54|
|18249735|       1557|           60|
|14989775|        614|           39|
|15073302|        526|           38|
|16075389|        591|           32|
|16606739|        678|            6|
|15705695|        431|            8|
|14576147|        817|           66|
|15134033|        944|           13|
|16551772|       1699|           47|
|17652157|       1407|           15|
|13089312|       1218|           52|
|13744500|       2232|          112|
|16829614|        738|           10|
|17524817|        328|            7|
+--------+-----------+-------------+
only showing top 20 rows



# Training data and testing data

In [12]:
train = pd.read_csv('../data/trainHistory.csv')
offers = pd.read_csv('../data/offers.csv')

In [13]:
# Add department to offers
dept = []
for i in range(offers.shape[0]):
    str_category = str(offers['category'].iloc[i])
    if len(str_category) == 4:
        dept.append(str_category[:2])
    else:
        dept.append(str_category[:1])
        
offers['dept'] = dept
offers['dept'] = pd.to_numeric(offers['dept'])

In [14]:
train = pd.merge(train, offers, how='left', on=['offer'])

In [15]:
# Rename columns
train.columns = ['id', 'chain', 'offer', 'market', 'repeattrips', 'repeater', 'offerdate', 
                 'offer_category', 'quantity', 'offer_company', 'offervalue', 'offer_brand',
                 'offer_dept']

In [16]:
# Reencode the target variable
train['repeater'] = np.where(train['repeater'] == 't', '1', '0')
train['repeater'] = pd.to_numeric(train['repeater'])

In [17]:
train['offerdate'] = train['offerdate'].astype(str)

In [18]:
test = pd.read_csv('../data/testHistory.csv')
test = pd.merge(test, offers, how='left', on=['offer'])

In [19]:
# Rename columns
test.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offer_category', 'quantity', 
                'offer_company', 'offervalue', 'offer_brand', 'offer_dept']

In [20]:
test['offerdate'] = test['offerdate'].astype(str)

In [21]:
train_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("repeattrips", LongType(), True),
    StructField("repeater", IntegerType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [22]:
train_df = sqlContext.createDataFrame(train, train_schema)

In [23]:
test_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [24]:
test_df = sqlContext.createDataFrame(test, test_schema)

In [25]:
# This function converts the string cell into a date:
stringToDate = F.udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

In [26]:
# Convert offerdate to date type
train_df = train_df.withColumn('offerdate', stringToDate(F.col('offerdate')))
test_df = test_df.withColumn('offerdate', stringToDate(F.col('offerdate')))

In [27]:
train_df = train_df.withColumnRenamed('repeater', 'label')

In [28]:
train_df = train_df.select('offer', 'label')
test_df = test_df.select('offer', 'id')

In [29]:
train_df.cache()
test_df.cache()

DataFrame[offer: bigint, id: bigint]

# Create pipeline

In [30]:
# Convert offers to numerical categories
offer_df = train_df.select('offer').union(test_df.select('offer'))
indexer = StringIndexer().setInputCol("offer").setOutputCol("offer_idx").fit(offer_df)

In [31]:
# One-hot-encoding of offer category
encoder = OneHotEncoder().setInputCol("offer_idx").setOutputCol("encoded").setDropLast(False)

# Assemble features
assembler = VectorAssembler(inputCols=["encoded"], outputCol="features")

# Logistic regression
lr = LogisticRegression(maxIter=10)

pipeline = Pipeline(stages=[indexer, encoder, assembler, lr])

# Cross-validation

In [32]:
regularization = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
kfolds = 2

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, regularization) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=kfolds)  # use 3+ folds in practice

In [33]:
cvModel = crossval.fit(train_df)

In [34]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(cvModel.bestModel.stages[3].coefficients))
print("Intercept: " + str(cvModel.bestModel.stages[3].intercept))

Coefficients: [-0.379723348063,0.0,0.0,0.0,0.768400355884,0.283072854182,0.724445673384,0.0,-1.49917011788,0.110476732692,0.0,0.533731203239,-1.09835919751,1.06045507859,-0.756703541547,0.0,0.0,0.207779814063,0.0,0.0,0.373975661609,0.0,-1.34356821348,-0.581562602664,0.0,-0.501814726175,-0.361011872975,-0.267339230459,0.0,0.0,-0.616881052971,-0.442098418788,-0.578534610908,-1.10035450515,-0.394207437488,-0.291895038351,-0.177066153567]
Intercept: -1.03268048499


# Predicting on training data 

In [35]:
train_pred = cvModel.transform(train_df)

In [36]:
train_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.67909319788162...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.83399965024132...|       0.0|
|[0.80414481437532...|       0.0|
|[0.85685173976960...|       0.0|
|[0.85685173976960...|       0.0|
|[0.71549092254498...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.85685173976960...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
|[0.67909319788162...|       0.0|
|[0.80414481437532...|       0.0|
|[0.80414481437532...|       0.0|
+--------------------+----------+
only showing top 20 rows



# Predicting on test data

In [37]:
test_pred = cvModel.transform(test_df)

In [38]:
test_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
|[0.73743523364202...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [39]:
test_id = test_df.select('id').toPandas()
test_prob = test_pred.select('probability').toPandas()

In [40]:
# I didn't know how to access the vector inside the probability column.
# To work around this, I looped through the probability column and used the indexing operations to pull out
# the individual probabilities.
prob = []
for i in range(test_prob.shape[0]):
    prob.append(test_prob['probability'][i][1])
test_prob['probYes'] = prob

In [41]:
test_final_pred = pd.merge(test_id, test_prob, left_index=True, right_index=True)

In [42]:
test_final_pred.head()

,id,probability,probYes
0,12262064,"[0.737435233642, 0.262564766358]",0.262565
1,12277270,"[0.737435233642, 0.262564766358]",0.262565
2,12332190,"[0.737435233642, 0.262564766358]",0.262565
3,12524696,"[0.737435233642, 0.262564766358]",0.262565
4,13074629,"[0.737435233642, 0.262564766358]",0.262565


In [43]:
test_final_pred['probYes'].describe()

count    151484.000000
mean          0.262785
std           0.018394
min           0.143148
25%           0.262565
50%           0.262565
75%           0.262565
max           0.434312
Name: probYes, dtype: float64